In [3]:
from tomok import IFCReader
from tomok import RuleUnitController

In [4]:
reader = IFCReader(ifc_filepath="/mnt/raid6/jaewook133/tomok/acc_server2/uploads/gAAAAABnRUhUhGYAPP9vbIqUPaQSIsf5OcTUKARvnfeyecZN9fcuVZBke-t1bm0GvcJrGzBXpWc_JQquwwuSxJG8jTzPBpt4lydn78pfziXox8CU5vAnwr82w9SQWaIvTmcnFPATrI8N04n7bCgTW6FuH-2JLO_3XG28YHxUDl5cOQetMrDMV64=")

In [5]:
for key in reader.subtype_product_cache.keys():
    print(key)

None
레벨:EXTREMO INICIAL 8 MM
NOTDEFINED
DOOR
HALF_TURN_STAIR
STRAIGHT
LANDING
STRINGER
OPENING
WINDOW
CURTAIN_PANEL
MULLION
FLOOR
STRAIGHT_RUN_STAIR
M_URINARIO-MURO-3D:M_URINARIO-MURO-3D
ROOF
CEILING
COLUMN
BEAM
BASESLAB


In [6]:
reader.subtype_product_cache['FLOOR']

[#58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.),
 #60671=IfcSlab('3fYD0Vq9XAsewjKrFiZwIE',#18,'바닥:exteriores:477652',$,'바닥:exteriores',#60653,#60670,'477652',.FLOOR.),
 #60705=IfcSlab('3fYD0Vq9XAsewjKrFiZwCM',#18,'바닥:exteriores:478796',$,'바닥:exteriores',#60690,#60704,'478796',.FLOOR.),
 #60729=IfcSlab('3fYD0Vq9XAsewjKrFiZxqZ',#18,'바닥:exteriores:479353',$,'바닥:exteriores',#60714,#60728,'479353',.FLOOR.),
 #60753=IfcSlab('3fYD0Vq9XAsewjKrFiZxzs',#18,'바닥:exteriores:479788',$,'바닥:exteriores',#60738,#60752,'479788',.FLOOR.),
 #60777=IfcSlab('3fYD0Vq9XAsewjKrFiZxxa',#18,'바닥:exteriores:480190',$,'바닥:exteriores',#60762,#60776,'480190',.FLOOR.),
 #60801=IfcSlab('3fYD0Vq9XAsewjKrFiZxWG',#18,'바닥:exteriores:480586',$,'바닥:exteriores',#60786,#60800,'480586',.FLOOR.),
 #60825=IfcSlab('3fYD0Vq9XAsewjKrFiZxkA',#18,'바닥:exteriores:480976',$,'바닥:exteriores',#60810,#60824,'480976',.FLOOR.),


In [7]:
reader.get_products()
e = reader.get_product_by_guid(guid="3fYD0Vq9XAsewjKrFiZqkg")
e

#62866=IfcSlab('3fYD0Vq9XAsewjKrFiZqkg',#18,'바닥:Hormigón 160 mm con losacero 50 mm:501488',$,'바닥:Hormigón 160 mm con losacero 50 mm',#62851,#62865,'501488',.FLOOR.)

In [11]:
e.tree

{'entity_instances': [[300119/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.3 (2)] [mat: None] [prof: None] [rule:None],
  [62872/IfcRelDefinesByProperties/] [pset: Pset_ReinforcementBarPitchOfSlab] [mat: None] [prof: None] [rule:None],
  [286044/IfcRelAssociatesClassification/A10] [pset: Uniformat] [mat: Uniformat] [prof: Uniformat] [rule:None],
  [300017/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.1 (3)] [mat: None] [prof: None] [rule:None],
  [62873/IfcRelDefinesByProperties/] [pset: Pset_SlabCommon] [mat: None] [prof: None] [rule:None],
  [300083/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 22_4.11.2 (5)] [mat: None] [prof: None] [rule:None],
  [300131/IfcRelDefinesByProperties/] [pset: KRPset_KDS 14 20 10_4.2.3 (2) ③] [mat: None] [prof: None] [rule:None],
  [285159/IfcRelDefinesByType/] [pset: None] [mat: None] [prof: None] [rule:None],
  [284440/IfcRelContainedInSpatialStructure/] [pset: None] [mat: None] [prof: None] [rule:None],
  [62871/Ifc

In [18]:
e['KRPset_KDS 14 20 22_4.11.2 (5)'].fIfck

AttributeError: fIfck property is not in #300082=IfcPropertySet('',$,'KRPset_KDS 14 20 22_4.11.2 (5)',$,(#300065,#300067,#300069,#300071,#300073,#300075,#300077,#300079,#300081)). ['설계 기준 (KDS)', '룰유닛 코드', '라이브러리 코드', '콘크리트에 의한 공칭전단강도', '콘크리트에 의한 공칭전단강도', '설계기준 압축강도', '위험단면의 둘레길이', '압축철근의 영향을 무시하고 계산된 슬래브 위험단면의 압축대 깊이의 평균값', '결과', 'KDS code', 'Ruleunit code', 'Library code', 'fOVc', 'fIVc', 'fIVc', 'fIbo', 'fIcu', 'Results']

In [1]:
from tomok import ACCController

controller = ACCController(
    resource_path="/mnt/raid6/jaewook133/tomok/acc_server2/temp_files",
    module_file_path="/mnt/raid6/jaewook133/tomok/acc_server2/temp_files/modules",
    rule_file="tree_temp2.csv"
)

controller.load_ifc_file("/mnt/raid6/jaewook133/tomok/acc_server2/uploads/gAAAAABnRUhUhGYAPP9vbIqUPaQSIsf5OcTUKARvnfeyecZN9fcuVZBke-t1bm0GvcJrGzBXpWc_JQquwwuSxJG8jTzPBpt4lydn78pfziXox8CU5vAnwr82w9SQWaIvTmcnFPATrI8N04n7bCgTW6FuH-2JLO_3XG28YHxUDl5cOQetMrDMV64=")

controller.set_subtype("FLOOR")
entities = controller.search_entities()
print("# of Entities:", len(entities))

controller.run_verification(7)

SCCs found: [['KDS241421_04010201_03'], ['KDS241421_04010201_01'], ['KDS241421_04010203_04', 'KDS241421_04010203_03'], ['KDS241421_04010202_03']]
SCCs found: []
SCCs found: []
SCCs found: [['KDS241421_04060201_03'], ['KDS241421_03010205_02_01']]
SCCs found: []
SCCs found: []
SCCs found: [['KDS142022_040101_01'], ['KDS142010_040203_02_03'], ['KDS142022_040201_01_01'], ['KDS142022_041103_02']]
SCCs found: [['KDS142022_040101_01'], ['KDS142010_040203_02_03'], ['KDS142022_041103_02'], ['KDS142022_041102_05'], ['KDS142022_041102_02']]
# of Entities: 21
LOGGING #58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.), KRPset_KDS 14 20 22_4.11.3 (1), fId
LOGGING #58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.), KRPset_KDS 14 20 22_4.11.3 (2), fIAv
LOGGING #58189=IfcSlab('2

[{'index': 0,
  'entity': #58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.),
  'ccc_results': [{'index': 0,
    'log': ['룰유닛 KDS142022_041103_01 실행',
     "Error executing code KDS142022_041103_01: KRPset_KDS 14 20 22_4.11.3 (1) property_set is not in #58189=IfcSlab('2f$SEp0bT4SfuPGVu0X6np',#18,'바닥:Hormigón 160 mm con losacero 50 mm:371309',$,'바닥:Hormigón 160 mm con losacero 50 mm',#58171,#58188,'371309',.FLOOR.). ['Pset_ReinforcementBarPitchOfSlab', 'Pset_EnvironmentalImpactIndicators', 'Pset_SlabCommon']"]},
   {'index': 1,
    'log': ['룰유닛 KDS142022_041102_01 실행',
     'Error executing code KDS142022_041102_01: None']},
   {'index': 2,
    'log': ['룰유닛 KDS142010_040203_02_03 실행',
     '입력 변수: {}',
     "API 반환 결과: {'fOphi': 0.75}"]},
   {'index': 3,
    'log': ['룰유닛 KDS142022_041103_02 실행',
     "Error executing code KDS142022_041103_02: KRPset_KDS 14 20 22_4.11.3 (2) p

In [9]:
ruleUnitController = RuleUnitController('ruleunits')
ruleUnitController.ruleunits

[]

In [10]:
import pprint
pprint.pprint(ruleUnitController.ruleunits_dict)

{}


In [11]:
ruleUnitController['kds']['142054']['040305_01']['KDS142054_040305_01']

KeyError: 'No such key: kds'

In [ ]:
from tomok import RuleIFC, IFCReader, Product, RuleUnitController


class RuleIFC_KDS142054_01(RuleIFC):
    # RuleIFC
    def __init__(self,
                 reader: IFCReader,
                 rule_units: RuleUnitController):
        self.reader = reader
        self.runit = {
            '01': rule_units['kds']['142054']['040305_01']['KDS142054_040305_01'],
            '03': rule_units['kds']['142054']['040305_03']['KDS142054_040305_03'],
            '04': rule_units['kds']['142054']['040305_04']['KDS142054_040305_04'],
            '05': rule_units['kds']['142054']['040305_05']['KDS142054_040305_05']
        }

    def retrieve_entities(self,
                          guid: str):
        self.reader.get_products() # IFC 파일에서 부재 목록을 읽어옵니다.
        try:
            target_entity = [self.reader.get_product_by_guid(guid)] # guid를 기반으로 부재를 검색합니다.
        except:
            return []
        return target_entity

    def _get_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str):
        try:
            return entity[krpset_name][en_name]
        except:                
            return -1
        
    def process(self,
                entity: Product):
        try:
            # KDS142054_040305_03
            # fOpsecNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
            #                             en_name="Correction factor for attached anchor groups in which eccentricity of tensile force acts")
            fIeprimN = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                        en_name="The distance between the resultant force of the tensile force acting on the anchor group subjected to tensile load and the centroid of the anchor group")
            fIcNa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                      en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsecNa = self.runit['03'].correction_factor_for_attached_anchor_groups_with_eccentricity_of_tensile_force(0, fIeprimN, fIcNa)
            # KDS142054_040305_04
            # fOpsedNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
            #                             en_name="Correction factor for edge influence of single anchor anchor or group of anchor anchors")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsedNa = self.runit['04'].correction_factor_for_edge_influence_of_single_or_group_of_anchor(0, fIcamin, fIcNa)
            # KDS142054_040305_05
            # fOpscpNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
            #                             en_name="Modification factors for bonded anchors used in uncracked concrete")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcac = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="risk podium distance")
            fOpscpNa = self.runit['05'].modification_factors_for_bonded_anchors_used_in_uncracked_concrete(0, fIcamin, fIcac)

            # KDS142054_040305_01
            fINa = self._get_p_value(entity=entity,
                                     krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                     en_name="Nominal bond strength of a single bonded anchor")
            fINag = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Nominal bond strength of adhesive anchor group")
            fIANa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Projected influence area of anchor group")
            fIANao = self._get_p_value(entity=entity,
                                       krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                       en_name="Projected influence area of a single anchor")
            fIpsedNa = fOpsedNa # KDS142054_040305_04 실행 결과
            fIpscpNa = fOpscpNa # KDS142054_040305_05 실행 결과
            fINba = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="Basic bond strength of a single bonded anchor in tension")
            fIpsecNa = fOpsecNa # KDS142054_040305_03 실행 결과
            fIn = self._get_p_value(entity=entity,
                                krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                en_name="Number of Attachable Anchors")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fIda = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The outer diameter of an anchor, or the shaft diameter of a head stud, head bolt, or claw bolt")
            fItauncr = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                        en_name="Characteristics Adhesion strength of adhesive anchors used in uncracked concrete")
            fIuserdefined = 1
            result = self.runit['01'].nominal_bond_strength_of_a_single_bonded_anchor(fINa,fINag,fIANa,fIANao,fIpsedNa,fIpscpNa,fINba,fIpsecNa,fIn,fIcNa,fIda,fItauncr,fIuserdefined)

            print(result)
            print("process end.")
        except:
            print("Fail")

In [ ]:
reader = IFCReader(ifc_filepath="ED_extension_usingToolkit_v2.ifc")
rule_units = RuleUnitController('ruleunits')
ruleifc = RuleIFC_KDS142054_01(reader=reader, rule_units=rule_units)

In [ ]:
entity = ruleifc.retrieve_entities("3s45s2y_vBWuTHL$ihTpyq")[0]

In [ ]:
print(entity["KRPset_KDS 14 20 54_4.3.5 (1)"]["The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength"])
print(ruleifc._get_p_value(entity, "KRPset_KDS 14 20 54_4.3.5 (1)", 'The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength'))

-
-


In [ ]:
entity = ruleifc.reader.get_products('IFCBearing')[109]
ruleifc.process(entity)

Pass
process end.


In [12]:
def build_KRP_string(rule_code: str):
            replacements = {
                "01": " ①",
                "02": " ②",
                "03": " ③",
                "04": " ④",
                "05": " ⑤",
                "06": " ⑥",
                "07": " ⑦",
                "08": " ⑧",
                "09": " ⑨",
            }
            temp = rule_code.split("_")
            template = "KRPset_{} {} {} {}_{} ({})"
            if len(temp[1]) > 6:
                code = template.format(
                    temp[0][:3],
                    temp[0][3:5],
                    temp[0][5:7],
                    temp[0][7:],
                    f"{temp[1][1]}.{temp[1][3]}.{temp[1][5]}.{temp[1][7]}",
                    temp[2][-1],
                )
            else:
                middle = [temp[1][i : i + 2] for i in range(0, len(temp[1]), 2)]
                middle = [str(int(x)) for x in middle]
                code = template.format(
                    temp[0][:3],
                    temp[0][3:5],
                    temp[0][5:7],
                    temp[0][7:],
                    ".".join(middle),
                    temp[2][-1],
                )
            if len(temp) == 4:
                code += replacements[temp[-1]]
            return code

In [13]:
build_KRP_string("KDS142022_041102_05")

'KRPset_KDS 14 20 22_4.11.2 (5)'